In [1]:
import torch
from IPython.display import Audio

In [2]:
from datasets import load_dataset
import datasets

In [3]:
cv13 = load_dataset('mozilla-foundation/common_voice_13_0', 'yue', trust_remote_code=True)
urban = load_dataset('danavery/urbansound8K')

In [4]:
speaker_dataset = cv13["test"].cast_column("audio", datasets.Audio(sampling_rate=16_000)).rename_column("client_id", "speaker_id")
noises_dataset = urban["train"].cast_column("audio", datasets.Audio(sampling_rate=16_000)).rename_column("classID", "noise_id")

In [5]:
speaker_dataset.set_format("pt")
noises_dataset.set_format("pt")

In [6]:
from MangoDemo.mangoPURE.data.mixer import DatasetMixer
from MangoDemo.mangoPURE.data.transforms import *

### Test single speaker with noise segment
Actually CreateRandomSpeaker is useless because the audio length is small, but... 
here is the example

In [26]:
trans = [
    CreateRandomSpeaker(speaker_dataset),
    AddRandomNoiseSegment(noises_dataset, no_noise_coef=False),
]
mixer = DatasetMixer(
    trans,
)

In [35]:
exm = mixer.generate()

In [36]:
Audio(exm.audio, rate=16000)

In [37]:
exm

MixedExample(audio=tensor([-1.4211e-14,  3.5527e-15,  3.7303e-14,  ...,  8.9697e-03,
         2.2352e-02,  1.4330e-02]), speakers_info=[SegmentInfo(start=0, length=70272, class_id='1ff3a60d65bf96e83951eb342a2a11ca800961f16257431d0a056dc91874926be22597bb1fc2a9ea367f8cc55e15d8edd8c8781fceea82de673270be0727b3ac')], noises_info=[SegmentInfo(start=15997, length=54275, class_id=tensor(4))])

### Test several speakers

In [11]:
trans = [
    CreateRandomBlankAudio(),
    AddSeveralRandomSpeakerSegments(speaker_dataset)
]
mixer = DatasetMixer(
    trans,
)

In [12]:
exm = mixer.generate()

In [13]:
Audio(exm.audio, rate=16000)

In [14]:
exm

MixedExample(audio=tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.2015, -0.1798, -0.1553]), speakers_info=None, noises_info=[SegmentInfo(start=103131, length=55296, class_id='8b4f94ddd6a2a65f1c7c57506fdd302ae94c72499cf9522cbdb3ebb086779cb68c1e5881bec962984cfac8f9803175eed7ed3b4bc4ec700f0f13890245c35c8f'), SegmentInfo(start=15023, length=65088, class_id='c4f354548c8d0fe251cbf9e4ab45c50112eaea9ac0de5b1fa2324b655c7bb21569a37b90e4049592bbd9835beab28f7a489770929146f1e58a568093d9241c48'), SegmentInfo(start=160779, length=40365, class_id='e484a627a8ba021b695f4e0fa47eb2abf44120c6a1875c516f13e2a52b2e04b2e7718f2da0ae8b7bcfcbdb1e68ff18f0a6529ebbd5d9225ec6ffebf2d4f85fbd'), SegmentInfo(start=170934, length=30210, class_id='f8be2b85141b7ba88ee91ddf11ae8672f3533530db991e0a4c5c29956651452d322dd74551dcff4e876169585df2944546482faca9c2515d25190a4082676a65')])

### Test several noises

In [15]:
trans = [
    CreateRandomBlankAudio(),
    AddSeveralRandomNoiseSegments(noises_dataset)
]
mixer = DatasetMixer(
    trans,
)

In [16]:
exm = mixer.generate()

In [17]:
Audio(exm.audio, rate=16000)

In [18]:
exm

MixedExample(audio=tensor([0., 0., 0.,  ..., 0., 0., 0.]), speakers_info=None, noises_info=[SegmentInfo(start=222103, length=64000, class_id=tensor(4)), SegmentInfo(start=76640, length=64000, class_id=tensor(1)), SegmentInfo(start=150843, length=64000, class_id=tensor(8))])

### Test filled noise

In [19]:
trans = [
    CreateRandomBlankAudio(),
    AddRandomFilledNoise(noises_dataset)
]
mixer = DatasetMixer(
    trans,
)

In [20]:
exm = mixer.generate()

In [21]:
Audio(exm.audio, rate=16000)

In [22]:
exm

MixedExample(audio=tensor([-0.0007, -0.0013, -0.0014,  ...,  0.0015,  0.0009,  0.0005]), speakers_info=None, noises_info=[SegmentInfo(start=0, length=287461, class_id=tensor(9))])